### Introduction to K-NN:
The k-Nearest Neighbors algorithm or KNN for short is a very simple technique.

The entire training dataset is stored. When a prediction is required, the k-most similar records to a new record from the training dataset are then located. From these neighbors, a summarized prediction is made.

Similarity between records can be measured many different ways. A problem or data-specific method can be used. Generally, with tabular data, a good starting point is the Euclidean distance.

Once the neighbors are discovered, the summary prediction can be made by returning the most common outcome or taking the average. As such, KNN can be used for classification or regression problems.

There is no model to speak of other than holding the entire training dataset. Because no work is done until a prediction is required, KNN is often referred to as a lazy learning method.

In [1]:
from math import sqrt
from random import seed, randrange
from csv import reader

In [2]:
def euclidean_distance(row1, row2):
    """
    Calculates the distance between any two records
    ###Input###
        row1 -> A row from the training data
        row2 -> A row from the test data
    ###Output###
        sqrt(dist) -> Returns the distance between the rows
    """
    dist = 0.0
    for i in range(len(row1) - 1): # Assuming last column being the target, excluding it
        dist += (row1[i] - row2[i])**2
    return sqrt(dist)

In [3]:
def get_neighbours(train, test_row, num_neighbours):
    """
    Calculates the required number of neighbours for a given test row
    ###Input### 
        train -> The entire training dataset
        test_row -> A row from the test dataset
        num_neighbours -> Number of neighbours we need
    ###Output### 
        neighbours -> Returns the neighbours for a given test row
    """
    distances = list()
    for train_row in train:
        dist = euclidean_distance(train_row, test_row)
        distances.append((train_row, dist))
    distances.sort(key = lambda x: x[1])
    neighbours = list()
    for i in range(num_neighbours):
        neighbours.append(distances[i][0])
    return neighbours

In [4]:
def predict_class(train, test_row, num_neighbours):
    """
    Determines the target class for a given test row
    ###Input### 
        train -> The entire training dataset
        test_row -> A row from the test dataset
        num_neighbours -> Number of neighbours we need
    ###Output### 
        prediction -> Returns the target class for a given test row
    """
    neighbours = get_neighbours(train, test_row, num_neighbours)
    outputs = [row[-1] for row in neighbours]
    prediction = max(set(outputs),key = outputs.count)
    return prediction

In [5]:
def load_csv(filename):
    """
    Loads the .csv file
    ###Input### 
        filename -> Name of the .csv file to be loaded
    ###Output### 
        dataset -> Returns the .csv file in the form of a python list
    """
    dataset = []
    with open(filename,"r") as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset

In [6]:
def str_column_to_float(dataset, column):
    """
    Changes the numeric column which is stored as strings to float dtype
    ###Input### 
        dataset -> The dataset containing the column
        column -> Index of the column which contains the numbers (stored as strings)
    ###Output### 
        None -> It changes the dtype of the column in the original dataset itself
    """
    for row in dataset:
        row[column] = float(row[column].strip())

In [7]:
def str_column_to_int(dataset, column):
    """
    Performs the label encoding
    ###Input### 
        dataset -> The dataset containing the column
        column -> Index of the column which contains the categories
    ###Output### 
        None -> It replaces the categories in a column to its numeric equivalent in the original dataset itself
    """
    classes = [row[column] for row in dataset]
    unique = set(classes)
    lookup = {}
    for i,value in enumerate(unique):
        lookup[value] = i
    print(lookup)
    for row in dataset:
        row[column] = lookup[row[column]]

In [8]:
def dataset_minmax(dataset):
    """
    Calculates the min and max values of all the columns
    ###Input### 
        dataset -> The entire dataset that needs to be scaled
    ###Output### 
        minmax -> A dictionary containing the min and max values of all the columns indexed 
                  by the column position
    """
    minmax = []
    for i in range(len(dataset[0])):
        col_values = [row[i] for row in dataset]
        min_val = min(col_values)
        max_val = max(col_values)
        minmax.append([min_val,max_val])
    return minmax

In [9]:
def normalize_dataset(dataset, minmax):
    """
    Performs the min-max normalization
    ###Input### 
        dataset -> The entire dataset that needs to be scaled
        minmax -> A dictionary containing the min and max values of all the columns indexed 
                  by the column position
    ###Output### 
        None -> It scales the dataset based on the minmax dictionary (Range of values : 0-1)
    """
    for col in range(len(dataset[0])):
        for row in dataset:
            row[col] = (row[col] - minmax[col][0])/(minmax[col][1] - minmax[col][0])

In [10]:
def cross_validation_split(dataset, nfolds):
    """
    Performs the splitting of the dataset into "nfolds" folds
    ###Input### 
        dataset -> The entire dataset that needs to be scaled
        nfolds -> Number of folds to be used for cross validation
    ###Output### 
        dataset_split -> A list containing "nfolds" folds to be used for cross validation
    """
    dataset_split = []
    dataset_copy = list(dataset)
    fold_size = int(len(dataset)/nfolds)
    for _ in range(nfolds):
        fold = []
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split

In [11]:
def accuracy_metric(actuals, predictions):
    """
    Calculates the accuracy of the predictions
    ###Input### 
        actuals -> Actual target values
        predictions -> Predicted target values
    ###Output### 
        acc -> Returns the accuracy of the predictions
    """
    correct = 0
    for i in range(len(actuals)):
        if actuals[i] == predictions[i]:
            correct += 1
    acc = (correct/len(actuals)) * 100.0
    return acc

In [12]:
def evaluate_algorithm(dataset, algorithm, nfolds, *args):
    """
    Evaluates any algorithm using the K-fold cross validation technique
    ###Input### 
        dataset -> Input dataset
        algorithm -> Function that performs K-NN and returns predictions
        nfolds -> Number of folds to be used in cross validation
        *args -> Variable number of arguments to be used in the function, if needed
    ###Output### 
        scores -> Returns a list of scores on the different folds used
    """
    folds = cross_validation_split(dataset, nfolds)
    print(len(folds))
    scores = []
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set,[])
        test_set = list(fold)
        predictions = algorithm(train_set, test_set, *args)
        actuals = [row[-1] for row in test_set]
        accuracy = accuracy_metric(actuals, predictions)
        scores.append(accuracy)
    return scores

In [13]:
def kNN(train_set, test_set, num_neighbours):
    """
    Performs K-NN classification by putting all the functions defined earlier together
    ###Input### 
        train_set -> training dataset
        test_set -> test dataset
        num neighbours -> Number of neighbours we need
    ###Output### 
        predictions -> Returns the predictions for the test set
    """
    predictions = []
    for row in test_set:
        prediction = predict_class(train_set, row, num_neighbours)
        predictions.append(prediction)
    return predictions

In [14]:
import os

In [15]:
os.getcwd()

'C:\\Users\\s5rxcy'

In [24]:
seed(1)
filename = 'iris.csv'
dataset = load_csv(filename)
dataset = dataset[1:]
for i in range(len(dataset[0])-1):
    str_column_to_float(dataset, i)
# convert class column to integers
str_column_to_int(dataset, len(dataset[0])-1)
# define model parameter
num_neighbors = 5
# define a new record
row = [4.5, 2.3, 1.3, 0.3]
# predict the label
label = predict_class(dataset, row, num_neighbors)
print('Data=%s, Predicted: %s' % (row, label))

{'Versicolor': 0, 'Virginica': 1, 'Setosa': 2}
Data=[4.5, 2.3, 1.3, 0.3], Predicted: 2


In [66]:
n_folds = 5
num_neighbors = 5
scores = evaluate_algorithm(dataset, kNN, n_folds, num_neighbors)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

5
Scores: [96.66666666666667, 100.0, 96.66666666666667, 96.66666666666667, 96.66666666666667]
Mean Accuracy: 97.333%


Lets check our output with the sklearn's implementation

In [36]:
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd

In [37]:
iris = pd.read_csv('iris.csv')

In [38]:
iris.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa


In [39]:
class_map = {'Setosa': 0, 'Versicolor': 1, 'Virginica': 2}

In [40]:
iris["variety"] = iris["variety"].map(class_map)

In [41]:
iris.dtypes

sepal.length    float64
sepal.width     float64
petal.length    float64
petal.width     float64
variety           int64
dtype: object

In [42]:
knn_classifier = KNeighborsClassifier(n_neighbors=5, metric = 'euclidean')

In [43]:
knn_classifier.fit(iris.drop("variety",axis = 1), iris["variety"])

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='euclidean',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [44]:
preds = knn_classifier.predict(iris.drop("variety",axis = 1))

In [45]:
accuracy_metric(iris.variety, preds)

96.66666666666667

In [46]:
from sklearn.model_selection import cross_val_score

In [59]:
scores = cross_val_score(knn_classifier, iris.drop("variety",axis = 1), iris["variety"], cv=5)

In [60]:
scores

array([0.96666667, 1.        , 0.93333333, 0.96666667, 1.        ])

In [61]:
scores.mean()

0.9733333333333334

The cross validation scores are a bit different because of the fact that different set of records being taken into different folds in two different cross validation techniques and by the tie-breaking mechanism between the two implementations